In [1]:
import pandas as pd
import cobra
import urllib

from cobra import Metabolite, Reaction

cobra_config = cobra.Configuration()
cobra_config.solver = "cplex"
# cobra_config.tolerance = 1e-6

In [2]:
url = "https://www.ebi.ac.uk/biomodels/model/download/MODEL1612130000.2?filename=MODEL1612130000_url.xml"
urllib.request.urlretrieve(url, f"../data/ebi_imt1026v3.xml") 
pichia, m_warnings = cobra.io.validate_sbml_model(f"../data/ebi_imt1026v3.xml")


COBRA warnings in validation, check error log for details.


In [3]:
rxns_pp_to_constrain = [
    'ATPM',
    'Ex_glc_D',
    'Ex_glyc',
    'Ex_tre',
    'LIPIDS_glyc',      # set inactive for glucose growth
    'PROTEINS_glyc',    #
    'STEROLS_glyc',     #
    'BIOMASS_glyc',     #
    'LIPIDS_meoh',      #
    'PROTEINS_meoh',    #
    'STEROLS_meoh',     #
    'BIOMASS_meoh',     #
    'STEROLS',          # activate for glucose growth
    'LIPIDS',           #
    'PROTEINS',         #
    'BIOMASS',
    "PHOSPHOLIPIDS",
    "PFK_3",
    "GLUK", # Block because hexo and glucokianse are the same, but harder to normalise solutions
    "G6PI",# Block because hexo and glucokianse are the same, but harder to normalise solutions
    "Ex_fab", 
    "G6PI3",# Block because hexo and glucokianse are the same, but harder to normalise solutions
    "RNA",
    "DNA", 
    "COF",
    "CARBOHYDRATES", 
    "ATPS3v",
    "Ex_cit", "Ex_etoh","Ex_abt_D", "Ex_abt","Ex_ac","Ex_acald","Ex_pyr", "SUCD1m", "PGI"]

for rxn in rxns_pp_to_constrain:
    pichia.reactions.get_by_id(rxn).lower_bound = 0
    pichia.reactions.get_by_id(rxn).upper_bound = 0
    
    if rxn in ['Ex_glc_D']:
        pichia.reactions.get_by_id(rxn).lower_bound = -2.5    #-0.33 original constrianed value
        pichia.reactions.get_by_id(rxn).upper_bound = 0           
        
    if rxn in ["PFK_3"]:
        pichia.reactions.get_by_id(rxn).bounds = (-1000, 0)
    if rxn in ["PGI"]:
        pichia.reactions.get_by_id(rxn).bounds = (0,1000)
    if rxn in ['ATPM']:
        pichia.reactions.get_by_id(rxn).upper_bound = 1000 
        pichia.reactions.get_by_id(rxn).lower_bound = 1.35


# 4. Add reactions to produce VHH as a model protein

- Protein Translation

In [4]:
reaction = Reaction('rvHHVAA')
pichia.add_reactions([reaction])
reaction.name = 'Translation of vHHV '
reaction.subsystem = 'Recombinant protein production'
reaction.lower_bound = 0 
reaction.upper_bound = 1000

vHHVAA_c = Metabolite(
    'vHHV_c',
    formula='C1639H2523N479O541S10', 
    name='Recombinant vHHV - amino acids',
    compartment='c'
    )
pichia.add_metabolites([vHHVAA_c])
rxn_compounds = [
    'h2o_c','atp_c','h_c','pi_c','adp_c','ala_L_c','arg_L_c','asn_L_c','asp_L_c','cys_L_c','gln_L_c','glu_L_c','gly_c', 'his_L_c',
    'ile_L_c','leu_L_c','lys_L_c','met_L_c','orn_c','phe_L_c','pro_L_c','ser_L_c','thr_L_c','trp_L_c','tyr_L_c','val_L_c','vHHV_c'
    ]
rxn_coefficients = [
    -31.524, -41.077, 41.077, 41.077, 41.077, -0.871, -0.528, -0.369, -0.449, -0.106, -0.422, -0.475, -1.425, -0.185,
    -0.238, -0.554, -0.211, -0.158, -0.000, -0.369, -0.317, -1.108, -0.607, -0.132, -0.343, -0.686, 1
    ]
reaction.add_metabolites(dict(zip(rxn_compounds, rxn_coefficients)))
pichia.groups.get_by_id("Recombinant protein production").add_members(pichia.reactions.rvHHVAA)

/home/users/bcoltman/.conda/envs/memo3.7/lib/python3.7/site-packages/cobra/core/group.py:91: UserWarning: need to pass in a list
  warn("need to pass in a list")


- Accumulation of vHHV 

In [5]:
reaction = Reaction('Ex_vHHV')
pichia.add_reactions([reaction])
reaction.name = 'Accumulation of vHHV '
reaction.subsystem = 'Recombinant protein production'
reaction.lower_bound = 0 
reaction.upper_bound = 1000

rxn_compounds = ['vHHV_c']                  
rxn_coefficients = [-1]

reaction.add_metabolites(dict(zip(rxn_compounds, rxn_coefficients)))
pichia.groups.get_by_id("Exchange reaction").add_members(pichia.reactions.Ex_vHHV)

Have to allow for phosphatidylgycerol transport to cytosol for biomass reaction. Not defined in the original model as not quantified/included in BIOMASS

In [6]:
reaction = Reaction("pgSCtm")
pichia.add_reactions([reaction])
reaction.name = 'Phosphatidylglycerol transport to cytosol for phospholipid composition'
reaction.subsystem = 'Transport, Mitochondrial'
reaction.lower_bound = -1000  # This is the default
reaction.upper_bound = 1000  # This is the default
reaction.add_metabolites({"pg_SC_m":-1, 
                          "pg_SC_c":1})
pichia.groups.get_by_id("Transport, Mitochondrial").add_members(pichia.reactions.pgSCtm)

Add sink reaction for glycerol

In [7]:
sink_name = "SK_glycogen_c"
metabolite = pichia.metabolites.get_by_id("glycogen_c")
reaction = Reaction(sink_name)
pichia.add_reactions([reaction])
reaction.name = f"{metabolite.name} Cytosolic Sink for storage 'Consumption'"
reaction.subsystem = "Sink, Cytosolic"
reaction.bounds = (0, 0)
reaction.add_metabolites({metabolite:-1})
pichia.groups.get_by_id("Other").add_members(pichia.reactions.SK_glycogen_c)

Unnecessary, but defined in Yeast8 model and not in imt1026. Instead, had a weird trapped futile ATP in the vacuole. Blocked and redefined this instead

In [8]:
reaction = Reaction("ATPS_v")
pichia.add_reactions([reaction])
reaction.name = 'ATPase, vacuolar'
reaction.subsystem = 'Transport, Vacuolar'
reaction.lower_bound = 0  # This is the default
reaction.upper_bound = 1000  # This is the default
reaction.add_metabolites({"h2o_c": -1.0, "atp_c": -1.0, "pi_c": 1.0, 
                          "adp_c": 1.0, "h_v": 1.0})
pichia.groups.get_by_id("Transport, Vacuolar").add_members(pichia.reactions.ATPS_v)

#### Define a scaled version of the consensus biomass equation

In [9]:
pichia.reactions.BIOMASS.id = "Consensus"
scaled_bio = pichia.reactions.Consensus.copy()
scaled_bio.id = "ScaledConsensus"
pichia.add_reactions([scaled_bio])

scaled_bio

macro_mets = [pichia.metabolites.get_by_id(x) for x in ["LIPIDS_c", "PROTEIN_c", "CARBOHYDRATES_c", "RNA_c", "DNA_c"]]

con_biomass_mets = pichia.reactions.ScaledConsensus.metabolites
con_bio_macros = {met: con_biomass_mets[met] for met in macro_mets}
total_gg = abs(sum(list(con_bio_macros.values())))
updated_con_bio_macros = {k:(v/total_gg)-v for k,v in con_bio_macros.items()}

pichia.reactions.ScaledConsensus.add_metabolites(updated_con_bio_macros, combine=True)



In [10]:
# imt = pichia.copy()
cobra.io.write_sbml_model(pichia, "../results/ModelWithProduction.xml")

In [11]:
glucose_mw = 180.156
with pichia as temp:
    qs = 1
    temp.objective = temp.reactions.Ex_vHHV
    temp.reactions.Ex_vHHV.bounds = (0,1000)
    temp.reactions.Ex_o2.bounds = (-1000,0)
    temp.reactions.Ex_co2.bounds = (0,1000)
    temp.reactions.Ex_glc_D.bounds = (-qs,-qs)
    temp.reactions.ATPM.bounds = (1.44, 1000)
    max_vhh = temp.slim_optimize()
    max_vhh_glc_yield = (max_vhh/(qs*(glucose_mw/1000)))
max_vhh_glc_yield

0.6041939140946085